In [ ]:
import subprocess as subp
import os

In [ ]:

#os.system => fuehre ein Befehl in die Shell aus
#subprocess.run => fuehre ein Befehl in die Shell aus
subp.run(['net', 'user', 'bob', 'smith', '/add'], shell=True)

os.system('net user bob smith /add')

In [ ]:
os.system('python --version')
subp.run(['python', '--version'])

## Benutzer erstellen

In [ ]:
import subprocess

def new_user():
    confirm = "N"
    while confirm != "Y":
        username = input("Gib den Namen des hinzuzufügenden Benutzers ein: ")
        print(f"Den Benutzernamen '{username}' verwenden? (Y/N)")
        confirm = input().upper()
    try:
          # 'Passwort123' is just a placeholder, set a real password
          subprocess.run(["net", "user", username, "Passwort123", "/add"], check=True, shell=True)
          print(f"Benutzer '{username}' wurde erfolgreich angelegt.")
    except subprocess.CalledProcessError:
          print("Fehler beim Anlegen des Benutzers. Vielleicht existiert er schon oder es fehlen Rechte.")

if __name__ == "__main__":
    new_user()

## Benutzer entfernen

In [ ]:
def del_user():
    confirm = "N"
    while confirm != "Y":
        username = input("Gib den Namen des zu löschenden Benutzers ein: ")
        print(f"Den Benutzernamen '{username}' verwenden? (Y/N)")
        confirm = input().upper()
    subprocess.run(["net","user"])
    try:
          # 'Passwort123' is just a placeholder, set a real password
          subprocess.run(["net", "user", username, "/delete"], check=True, shell=True)
          print(f"Benutzer '{username}' wurde erfolgreich gelöscht.")
    except subprocess.CalledProcessError:
          print("Fehler beim Loeschen des Benutzers. Vielleicht existiert er schon oder es fehlen Rechte.")

if __name__ == "__main__":
    del_user()

## Benutzer zu Gruppen Hinzufuegen Linux Variante

In [ ]:
import subprocess

def benutzer_zur_gruppe_hinzufuegen():
    benutzername = input("Gib den Namen des Benutzers ein, den du zu einer Gruppe hinzufügen möchtest: ")
    output = subprocess.Popen('groups', stdout=subprocess.PIPE).communicate()[0]

    print("Gib eine Liste von Gruppen ein, zu denen der Benutzer hinzugefügt werden soll")
    print("Die Liste sollte durch Leerzeichen getrennt sein, z. B.:\r\n gruppe1 gruppe2 gruppe3")
    print("Verfügbare Gruppen sind:\r\n " + output.decode())

    ausgewaehlteGruppen = str(input("Groups: "))

    output = output.split(" ")

    #Vor Split:  "gruppe1 gruppe2 gruppe3"
    #Nach Split: ["gruppe1", "gruppe2", "gruppe3"]
    ausgewaehlteGruppen = ausgewaehlteGruppen.split(" ")

    print("Hinzufügen zu:")
    gefunden = True
    gruppenString = ""
    for grp in ausgewaehlteGruppen:
        for vorhandeneGruppe in output:
            if grp == vorhandeneGruppe:
                gefunden = True
                print("- Vorhandene Gruppe: " + grp)
                gruppenString = gruppenString + grp + ","
        if gefunden == False:
            print("- Neue Gruppe: " + grp)
            gruppenString = gruppenString + grp + ","
        else:
            gefunden = False
    gruppenString = gruppenString[:-1] + " "
    bestaetigung = ""
    while bestaetigung != "Y" and bestaetigung != "N":
        print("Benutzer '" + benutzername + "' zu diesen Gruppen hinzufügen? (Y/N)")
        bestaetigung = input().upper()
    if bestaetigung == "N":
        print("Benutzer '" + benutzername + "' wurde nicht hinzugefügt")
    elif bestaetigung == "Y":
        os.system("sudo usermod -aG " + gruppenString + benutzername)
        #subprocess.run(["newgrp", gruppenString + benutzername])
        print("Benutzer '" + benutzername + "' wurde hinzugefügt")


## Benutzer zu Gruppe hinzufuegen Windows Variante

In [ ]:
import subprocess
def benutzer_zur_gruppe_hinzufuegen():
    # Benutzername abfragen
    benutzername = input("Gib den Namen des Benutzers ein, den du zu einer Gruppe hinzufügen möchtest: ")
    # PowerShell-Befehl zum Abrufen aller lokalen Gruppen
    get_groups_command = "Get-LocalGroup | Select-Object -ExpandProperty Name"
    result = subprocess.run(["powershell", "-Command", get_groups_command], capture_output=True, text=True)
    output = result.stdout.strip().splitlines()  # Liste der vorhandenen Gruppen
    # Gruppenliste vom Benutzer abfragen
    print("Gib eine Liste von Gruppen ein, zu denen der Benutzer hinzugefügt werden soll")
    print("Die Liste sollte durch Leerzeichen getrennt sein, z. B.: gruppe1 gruppe2 gruppe3")
    print("Verfügbare Gruppen sind:\n" + "\n".join(output))
    ausgewaehlteGruppen = input("Gruppen: ").split()
    print("Hinzufügen zu:")
    gruppenString = ""
    for grp in ausgewaehlteGruppen:
        if grp in output:
            print("- Vorhandene Gruppe: " + grp)
        else:
            print("- Neue Gruppe: " + grp)
        gruppenString += grp + " "
    # Bestätigung abfragen
    bestaetigung = ""
    while bestaetigung != "Y" and bestaetigung != "N":
        print(f"Benutzer '{benutzername}' zu diesen Gruppen hinzufügen? (Y/N)")
        bestaetigung = input().upper()
    if bestaetigung == "N":
        print(f"Benutzer '{benutzername}' wurde nicht hinzugefügt")
    elif bestaetigung == "Y":
        # Benutzer zu jeder Gruppe hinzufügen
        for grp in ausgewaehlteGruppen:
            add_command = f"Add-LocalGroupMember -Group '{grp}' -Member '{benutzername}'"
            subprocess.run(["powershell", "-Command", add_command], shell=True)
        print(f"Benutzer '{benutzername}' wurde hinzugefügt")